Pip install

In [1]:
pip install momepy

     |████████████████████████████████| 235kB 9.5MB/s 
     |████████████████████████████████| 2.4MB 15.7MB/s 
     |████████████████████████████████| 972kB 50.0MB/s 
     |████████████████████████████████| 6.5MB 49.6MB/s 
     |████████████████████████████████| 14.8MB 336kB/s 
  Created wheel for libpysal: filename=libpysal-4.3.0-py2.py3-none-any.whl size=2379082 sha256=bae3b7a2ca817efe898126eb410365d5436f176665f61f9d7e2f096b52adc00e
  Stored in directory: /root/.cache/pip/wheels/68/44/db/3d7a8b7d9ae5dba0cfcfb37f14e2a6e6f26b03557bdf1cf06d
Successfully built libpysal


Imports

In [2]:
import json
import networkx as nx
from networkx.readwrite import json_graph
import momepy as mm

Preprocessing

In [3]:
arithmetic=["add","sub","inc","dec","div","mul"]
shift=["sal","shl","sar","shr","psl","psr"]
four=["cmov","idiv","imul","pxor","punp","padd","pand","psub","pand"]

def decide_class(str):
  if(str[0:3]=="mov"):
    if(len(str)>4):
      if(str[3]!="s" and str[3]!="z"):
        return str[0:3]
      else:
        return str[0:4]
    else:
      return str[0:3]
  elif(str[0:3]=="cvt" or str[-2:]=="pd" or str[-2:]=="sd"):
    return "float"
  elif(str[0:3]in arithmetic):
    #return "math"
    return str[0:3]
  elif(str[0:3]in shift):
    return "shift"
  elif(str[0:3]=="and"):
    return "and"
  elif(str[0:3]=="cmp" or str[0:4]=="pcmp"):
    return "cmp"
  elif(str[0]=='j'):
    if(str[0:3]!='jmp'):
      return "jumpc"
    else:
      return str[0:3]
  elif(str[0:4] in four):
    return str[0:4]
  elif(str[0:4]=="cmov"):
    return str[0:4]
  elif(str[0:3]=="xor"):
    return "xor"
  elif(str[0:3]=="set"):
    return str[0:3]
  elif(str[0]=='f' and len(str)>=3):
    #this is an assumpion (FPU)
    return "float"
  return str

file=open("drive/My Drive/ML/HW1/dataset.json","r")
newfile=open("drive/My Drive/ML/HW1/processed_dataset.json","w")

for line in file:
  json_data = json.loads(line)
  string=""
  list=json_data['lista_asm']
  cfg=json_data['cfg']
  graph = json_graph.adjacency_graph(cfg)
  #nx.draw(graph)
  #print(nx.info(graph))
  edges=nx.number_of_edges(graph)
  nodes=nx.number_of_nodes(graph)
  has_cycles=1
  try: nx.algorithms.cycles.find_cycle(graph)
  except: has_cycles=0
  cyclo_complexity=mm.cyclomatic(graph,radius=None)
  flag=False
  for instruction in list.split("'"):
    if(flag==False):
      flag=True
      continue
    elif(flag==True):
      flag=False
    inst_list=instruction.split(" ")
    inst=""
    l=len(inst_list)
    if(l>1):
      inst=inst_list[0]
      if(inst!=""):
        string=string+"'"+decide_class(inst)+"',"
      if(inst_list[1][:3]=="xmm"):
        inst=inst_list[1][:3]
        string=string+"'"+inst+"',"
      if(inst_list[1][:4]=="byte"):
        string=string+"'"+"byte"+"',"
      if(inst_list[1][:5]=="dword" or inst_list[1][:5]=="qword"):
        string=string+"'"+inst_list[1][:5]+"',"
    if(l>2 and inst_list[2][:4]=="byte"):
      string=string+"'"+"byte"+"',"
    else:
      inst=inst_list[0]
      if(inst!=""):
        string=string+"'"+decide_class(inst)+"',"

  finalstr='{"id": '+'"'+json_data['id']+'", '+'"semantic": "'+json_data['semantic']+'", '+'"edges": "'+str(edges)+'", '+'"cycles": "'+str(has_cycles)+'", '+'"cyclomatic": "'+str(cyclo_complexity)+'", '+'"instructions": "['+string[0:-1]+']"}'
  newfile.write(finalstr+'\n')
file.close()
newfile.close()

print("processed dataset saved in processed_dataset.json !")

processed dataset saved in processed_dataset.json !
